# Clean DataSets 

## 0.Import origin data

In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass  # Para obtener la contraseña sin mostrar la entrada

matchs = pd.read_csv("C:/Users/osanc/Desktop/IronHack/Sem_4/Project/england-premier-league-matches-2018-to-2019-stats.csv")
players = pd.read_csv("C:/Users/osanc/Desktop/IronHack/Sem_4/Project/england-premier-league-players-2018-to-2019-stats.csv")
referes = pd.read_csv("C:/Users/osanc/Desktop/IronHack/Sem_4/Project/england-premier-league-referes-2018-to-2019-stats.csv")
stadiums = pd.read_csv("C:/Users/osanc/Desktop/IronHack/Sem_4/Project/england-premier-league-stadiums-2018-to-2019-stats.csv")
teams = pd.read_csv("C:/Users/osanc/Desktop/IronHack/Sem_4/Project/england-premier-league-teams-2018-to-2019-stats.csv")

## 1.Clean Teams 

In [ ]:
# 1.1 Crear una columna con un ID  Teams
teams['id_teams'] = ['T{:03d}'.format(i) for i in range(1, len(teams) + 1)]

In [ ]:
# 1.2 Seleccionar y orden de columnas de interes para análisis
teams = teams[['id_teams','team_name', 'common_name', 'matches_played', 'matches_played_home',
       'matches_played_away','wins', 'wins_home',
       'wins_away', 'draws', 'draws_home', 'draws_away', 'losses',
       'losses_home', 'losses_away', 'league_position', 'average_possession',
       'average_possession_home', 'average_possession_away']]

## 2.Clean Players

In [ ]:
# 2.1 Selección de columnas de interes - players

players = players[['full_name',
 'age',
 'birthday_GMT',
 'position',
 'Current Club',
 'minutes_played_overall',
 'minutes_played_home',
 'minutes_played_away',
 'nationality',
 'appearances_overall',
 'appearances_home',
 'appearances_away',
 'goals_overall',
 'goals_home',
 'goals_away',
 'assists_overall',
 'assists_home',
 'assists_away',
 'penalty_goals',
 'penalty_misses',
 'yellow_cards_overall',
 'red_cards_overall'
 ]]

In [ ]:
# 2.2 Unificación con teams para remplazar current club de players por id_teams de teams
players = pd.merge(players, teams, left_on='Current Club', right_on='common_name', how='left')

In [ ]:
# 2.3 Creacion de codigo unico "id_player" en players
players['id_player'] = ['P{:03d}'.format(i) for i in range(1, len(players) + 1)]

In [ ]:
# 2.4 Final selección y orden de columnas de interes
players = players[["id_player",'full_name', 'age', 'birthday_GMT', 'position','id_teams' ,
       'minutes_played_overall', 'minutes_played_home', 'minutes_played_away',
       'nationality', 'appearances_overall', 'appearances_home',
       'appearances_away', 'goals_overall', 'goals_home', 'goals_away',
       'assists_overall', 'assists_home', 'assists_away', 'penalty_goals',
       'penalty_misses', 'yellow_cards_overall', 'red_cards_overall']]

## 3.Clean Stadiums

In [ ]:
## 3.1 Creacion de codigo unico para estadios id_stadium
stadiums['id_stadium'] = ['S{:03d}'.format(i) for i in range(1, len(stadiums) + 1)]

In [ ]:
## 3.2 Seleccion y orden de columnas de interes
stadiums = stadiums[['id_stadium','Stadium(city)', 'Stadium', 'location', 'ability', 'opening_year',
       ]]

## 4. Clean Referes

In [ ]:
# 4.1 Creación de codigo unico id_refere
referes['id_refere'] = ['R{:03d}'.format(i) for i in range(1, len(referes) + 1)]

In [ ]:
# 4.2 Selección y orden de variables de interes
referes = referes[['id_refere','Name', 'Nationality', 'Birthdate', 'debut_year']]

## 5. Clean Calendary

In [ ]:
# 5.1 Extraccion de columna date_GMT de match para creacion de calendary
calendary = matchs[['date_GMT']]

In [ ]:
# 5.2 Extracción de solo fechas de columna
def extract_date_only(date_string):
    return pd.to_datetime(date_string, format='%b %d %Y - %I:%M%p').date()


calendary["date"] = calendary['date_GMT'].apply(extract_date_only)

In [ ]:
# 5.3 Creacion de year, month y day 
calendary['year'] = pd.to_datetime(calendary['date']).dt.year
calendary['month'] = pd.to_datetime(calendary['date']).dt.month
calendary['day'] = pd.to_datetime(calendary['date']).dt.day

In [ ]:
# 5.4 Eliminacion de duplicados
calendary = calendary.drop_duplicates(subset='date_GMT')

In [ ]:
# 5.5 Creacion de codigo unico ID de calendary
calendary['id_calendary'] = ['C{:03d}'.format(i) for i in range(1, len(calendary) + 1)]

In [ ]:
# 5.6 Selección y orden  de columnas de interes
calendary = calendary[['id_calendary','date_GMT', 'date', 'year', 'month', 'day']]

## 6. Clean Matchs

In [ ]:
# 6.1 Selección de columnas de interes - matchs
matchs = matchs[['date_GMT', 'home_team_name',
       'away_team_name', 'referee', 'Game Week','home_team_goal_count',
       'away_team_goal_count', 'total_goal_count','home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards','stadium_name'
 ]]

In [ ]:
# 6.2 Creación de código unico de partido id_match
matchs['id_match'] = ['M{:03d}'.format(i) for i in range(1, len(matchs) + 1)]

In [ ]:
# 6.3 Merge con tabla refere para remplazar refere de matchs por id_refere de referes
matchs = pd.merge(matchs, referes, left_on='referee', right_on='Name', how='left')

In [ ]:
# 6.4 Merge con tabla stadium para remplazar stadium_name de matchs por id_stadium de stadium
matchs = pd.merge(matchs, stadiums, left_on='stadium_name', right_on='Stadium(city)', how='left')

In [ ]:
## 6.5 Merge con tabla calendary para remplazar date_MGT de matchs por id_calendary de calendary
matchs = pd.merge(matchs, calendary, on = "date_GMT" ,how='left')

In [ ]:
## 6.6 Primera selección y orden de columnas 
matchs = matchs[['id_match','id_calendary','id_stadium','id_refere','home_team_name', 'away_team_name', 'referee', 'Game Week',
       'home_team_goal_count', 'away_team_goal_count', 'total_goal_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards']]

In [ ]:
## 6.7 Merge con tabla teams para remplazar home_team_name de matchs por id_team de teams 
matchs = pd.merge(matchs, teams, left_on='home_team_name', right_on='common_name', how='left')

In [ ]:
## 6.8 Segunda selección y orden de columnas 
matchs = matchs[['id_match', 'id_calendary', 'id_stadium', 'id_refere', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'home_team_yellow_cards',
       'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards',
       'id_teams', 'team_name', 'common_name']]

In [ ]:
## 6.9 Reemplazo de nombres de columnas columnas home
matchs.rename(columns={"common_name":"home_team","id_teams":"id_teams_home"}, inplace= True)

In [ ]:
## 6.10 Tercera selección y orden de columnas
matchs = matchs[['id_match', 'id_calendary', 'id_stadium', 'id_refere', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'home_team_yellow_cards',
       'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards',
       'id_teams_home', 'home_team']]

In [ ]:
## 6.11 Merge con tabla teams para remplazar away_team_name de matchs por id_team de teams 
matchs = pd.merge(matchs, teams, left_on='away_team_name', right_on='common_name', how='left')

In [ ]:
## 6.12 Reemplazo de nombres de columnas columnas home
matchs.rename(columns={"common_name":"away_team","id_teams":"id_teams_aways"}, inplace= True)

In [ ]:
## 6.13 Tercera selección y orden de columnas
matchs = matchs[['id_match', 'id_calendary', 'id_stadium', 'id_refere','id_teams_home', 'id_teams_aways',
       'Game Week', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'home_team_yellow_cards',
       'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards']]

In [ ]:
## 6.14 Cambio obede a stadiums pero tuvo que realizarse al final de la limpieza 
stadiums = stadiums[['id_stadium', 'Stadium', 'location', 'ability',
       'opening_year']]

## 7.Export to Mysql

In [ ]:
# Obtener la contraseña de forma segura
password = getpass.getpass()
# Configuración de la base de datos y la conexión
bd = "proj_soccer"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)
players.to_sql('players', con=engine, if_exists='replace', index=False)
matchs.to_sql('matchs', con=engine, if_exists='replace', index=False)
calendary.to_sql('calendary', con=engine, if_exists='replace', index=False)
referes.to_sql('referes', con=engine, if_exists='replace', index=False)
stadiums.to_sql('stadiums', con=engine, if_exists='replace', index=False)
teams.to_sql('teams', con=engine, if_exists='replace', index=False)